In [0]:
import pandas as pd
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')


In [0]:

def getCrumb(ticker):
    URL  = "https://finance.yahoo.com/quote/%s/history" % ticker
    import requests 
    import re
    r = requests.get(url = URL) 
    content = str(r.content)
    r1 = re.findall(r"CrumbStore\":\{\"crumb\":\"[A-Za-z0-9.\\\-]+\"",content)[0]
    crumb = r1.replace("CrumbStore\":{\"crumb\":\"","").replace('"',"")
    return crumb,r.cookies


def get_data(symbol, data_type, start_date , end_date):
    try:
        import requests 
        import re
        from io import StringIO
        import os
        import os.path
        
        directory = "drive/My Drive/Investment/data"
        filename = directory+"/"+symbol+"_"+data_type
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        #check cache
        last_saved = 86401
        if os.path.isfile(filename):
            statbuf = os.stat(filename)
            last_saved = datetime.now().timestamp() - statbuf.st_mtime
             
        if(last_saved>(86400)):
            #crawl
            api_url = "https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2={2}&interval={3}&events={4}&crumb={5}"
            (crumb, cookies)=getCrumb(symbol)
            api_url = api_url.format(symbol,0,datetime.now().strftime('%s'),"1d", data_type, crumb)
            r = requests.get(url = api_url, cookies = cookies.get_dict(".yahoo.com"))  
            content = str(r.content).replace(",",'","').replace("\\n",'"\r\n"').replace("b\'","").replace("'","")
            res = '"'+content+'"'
            
            #save
            text_file = open(filename, "w")
            n = text_file.write(r.content.decode('utf-8'))
            text_file.close()
        
        #load
        df = pd.read_csv(filename)
        df["Date"]=pd.to_datetime(df['Date'])
        df = df[df["Date"]>=start_date][df["Date"]<=end_date]
        return df  
    except:
        print("Error getting data")

def get_price_data(stock, start_date, end_date):
    print("Getting price data ...")
    price = get_data(stock,"history", start_date, end_date)
    if(price is not None):
        price.rename(columns={'Date':'date','Open':'open','High':'high',
                              'Low':'low','Close':'close','Adj Close':'adjClose','Volume':'vol'}, inplace=True)
        price["date"]=pd.to_datetime(price['date'])
        price = price.set_index(["date"])
        return price
def get_dividend_data(stock, start_date, end_date):
    print("Getting dividend data ...")
    dividend = get_data(stock,"div", start_date, end_date)
    if(dividend is not None):
        dividend.rename(columns={'Date':'date','Dividends':'dividend'}, inplace=True)
        dividend["date"]=pd.to_datetime(dividend['date'])
        dividend = dividend.set_index(["date"])
        return dividend

In [0]:
stocks = {
    # Bank
    "D05.SI":"DBS Group Holdings Ltd",
    "O39.SI":"Oversea-Chinese Banking Corporation Limited",
    "U11.SI":"United Overseas Bank Limited",
    
    # REIT
    "XZL.SI":"ARA US Hospitality Trust",
    "A17U.SI":"Ascendas Reit",
    "BMGU.SI":"BHG Retail Reit",
    "K2LU.SI":"Cache Log Trust",
    "J91U.SI":"ESR REIT",
    "C61U.SI":"CapitaCom Trust",
    "C38U.SI":"CapitaMall Trust",
    "CNNU.SI":"Cromwell European",
    "J85.SI":"CDL HTrust",


    "O5RU.SI":"AIMS APAC Reit",
    "LIW.SI":"Eagle US Hospitality Trust",
    "BWCU.SI":"EC World REIT",
    "MXNU.SI":"Elite Commercial REIT",
    "Q5T.SI":"Far East HTrust",
    "AW9U.SI":"First Reit",
    "ND8U.SI":"Frasers Com Tr",
    "J69U.SI":"Frasers Cpt Tr",
    "ACV.SI":"Frasers HTrust",
    "BUOU.SI":"Frasers Log & Ind Tr",
    "UD1U.SI":"IREIT Global",
    "AJBU.SI":"Keppel DC Reit",
    "CMOU.SI":"Keppel Pacific Oak US REIT",
    "K71U.SI":"Keppel Reit",
    "JYEU.SI":"LendLease Global Comm",
    "D5IU.SI":"Lippo Mall Indo Retail Trust",
    "BTOU.SI":"ManulifeReit USD",
    "RW0U.SI":"Mapletree North Asia Comm Tr",
    "ME8U.SI":"Mapletree Ind Tr",
    "M44U.SI":"Mapletree Log Tr",
    "N2IU.SI":"Mapletree Comm Tr",
    "TS0U.SI":"OUE Com Reit",
    "C2PU.SI":"ParkwayLife Reit",
    "OXMU.SI":"Prime US REIT",
    "M1GU.SI":"Sabana Reit",
    "CRPU.SI":"Sasseur REIT",
    "SV3U.SI":"SoilbuildBizReit",
    "SK6U.SI":"SPHREIT",
    "P40U.SI":"StarhillGbl Reit",
    "T82U.SI":"Suntec Reit",
    
    #
    "Z77.SI":"Singtel",
    "BN4.SI":"Keppel Corporation Limited",
    "C31.SI":"CapitaLand Limited",
    "C6L.SI":"Singapore Airlines Limited",
    
    # USA
    "QQQ":"Invesco QQQ Trust",
    "AAPL":"Apple Inc",
    "MSFT":"Microsoft Corporation",
    "TSLA":"Tesla, Inc",
    "GOOG":"Alphabet Inc",
    "FB":"Facebook",
    "NVDA":"NVIDIA Corporation",
    "AMZN":"Amazon",
    "V":"Visa Inc",
    "MA":"Mastercard Incorporated",
    "FB":"Facebook",
    "DAL":"Delta Air Lines, Inc",
    "UAL":"United Airlines Holdings Inc",
    "AAL":"American Airlines Group Inc",
    "PYPL":"Paypal Holdings Inc",
    "PYPL":"Paypal Holdings Inc",
    "XOM":"Exxon Mobil Corporation",
    "CVX":"Chevron Corporation",
    "AMD":"Advanced Micro Devices, Inc.",
    "C":"Citigroup Inc.",
    "GS":"Goldman sach",
    "JPM":"JP Morgan",
    "BAC":"Bank of America",
    "MS":"Morgan Stanley",
    	
    #mREIT
    "IVR":"Invesco Mortgage Capital Inc",
    "MFA":"MFA Financial, Inc",
    "NYMT":"New York Mortgage Trust, Inc",
    "MITT":"AG Mortgage Investment Trust, Inc",
    "XAN":"Exantas Capital Corp",
    
}

In [0]:
#crawl data from yahoo

data = None
for s in stocks:
    stock = s
    df = get_price_data(stock, datetime(2019,1,1),datetime(2020,5,30))
    MA = 200
    df['MA'] = df.rolling(window=MA).mean()["close"]
    period = 60
    peak = df[-period:]["close"].max()
    dip = df[-period:]["close"].min()
    down = (peak-dip)/peak
    up = (peak-dip)/dip
    df["MA"] = df[-period:]["MA"].max()
    df["current%"] = ((df["close"]-dip)/dip)*100
    df["MA%"] = ((df["MA"]-dip)/dip)*100
    df["peak"] = peak
    df["peak%"] = up*100
    df["dip"] = dip
    df["code"] = stock
    df["stock"] = stocks[stock]
    df["5%"] = df["dip"]*1.05
    df["10%"] = df["dip"]*1.1
    df["15%"] = df["dip"]*1.15
    df["20%"] = df["dip"]*1.2
    df["25%"] = df["dip"]*1.25
    df["eta_growth_peak%"] = df["peak%"]-df["current%"]
    df["eta_growth_MA%"] = df["MA%"]-df["current%"]
    res = df[-1:][["stock","code", "current%", 
                   #"peak%", "MA%", 
                   "eta_growth_peak%","eta_growth_MA%",
                   "dip", "close", 
                   "5%", "10%", "15%", "20%", "25%"]]
    if(data is None):
        data = res
    else:
        None
        data= pd.concat([data, res])

pd.set_option('display.max_rows', None)
data

Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting price data ...
Getting pri

,stock,code,current%,eta_growth_peak%,eta_growth_MA%,dip,close,5%,10%,15%,20%,25%
date,,,,,,,,,,,,
2020-04-30,DBS Group Holdings Ltd,D05.SI,18.246447,34.004753,33.361679,16.879999,19.959999,17.723999,18.567999,19.411999,20.255999,21.099999
2020-04-30,Oversea-Chinese Banking Corporation Limited,O39.SI,15.749040,26.248399,25.666453,7.810000,9.040000,8.200500,8.591000,8.981500,9.372000,9.762500
2020-04-30,United Overseas Bank Limited,U11.SI,15.765509,34.092203,31.832669,17.570000,20.340000,18.448500,19.327000,20.205500,21.084000,21.962500
2020-04-30,ARA US Hospitality Trust,XZL.SI,39.344262,144.262295,144.959016,0.305000,0.425000,0.320250,0.335500,0.350750,0.366000,0.381250
2020-04-30,Ascendas Reit,A17U.SI,29.694323,20.960699,3.276563,2.290000,2.970000,2.404500,2.519000,2.633500,2.748000,2.862500
2020-04-30,BHG Retail Reit,BMGU.SI,61.538462,14.102564,16.769231,0.390000,0.630000,0.409500,0.429000,0.448500,0.468000,0.487500
2020-04-30,Cache Log Trust,K2LU.SI,50.000000,47.222222,54.291667,0.360000,0.540000,0.378000,0.396000,0.414000,0.432000,0.450000
2020-04-30,ESR REIT,J91U.SI,41.666667,87.500000,80.800960,0.240000,0.340000,0.252000,0.264000,0.276000,0.288000,0.300000
2020-04-30,CapitaCom Trust,C61U.SI,21.052632,36.090226,34.093985,1.330000,1.610000,1.396500,1.463000,1.529500,1.596000,1.662500


In [0]:
d = data
d = d.loc[d["eta_growth_peak%"]>30]
#d = d.loc[d["current%"]<30]
d

,stock,code,current%,eta_growth_peak%,eta_growth_MA%,dip,close,5%,10%,15%,20%,25%
date,,,,,,,,,,,,
2020-04-30,DBS Group Holdings Ltd,D05.SI,18.246447,34.004753,33.361679,16.879999,19.959999,17.723999,18.567999,19.411999,20.255999,21.099999
2020-04-30,United Overseas Bank Limited,U11.SI,15.765509,34.092203,31.832669,17.570000,20.340000,18.448500,19.327000,20.205500,21.084000,21.962500
2020-04-30,ARA US Hospitality Trust,XZL.SI,39.344262,144.262295,144.959016,0.305000,0.425000,0.320250,0.335500,0.350750,0.366000,0.381250
2020-04-30,Cache Log Trust,K2LU.SI,50.000000,47.222222,54.291667,0.360000,0.540000,0.378000,0.396000,0.414000,0.432000,0.450000
2020-04-30,ESR REIT,J91U.SI,41.666667,87.500000,80.800960,0.240000,0.340000,0.252000,0.264000,0.276000,0.288000,0.300000
2020-04-30,CapitaCom Trust,C61U.SI,21.052632,36.090226,34.093985,1.330000,1.610000,1.396500,1.463000,1.529500,1.596000,1.662500
2020-04-30,CapitaMall Trust,C38U.SI,24.342105,42.105263,44.036184,1.520000,1.890000,1.596000,1.672000,1.748000,1.824000,1.900000
2020-04-30,Cromwell European,CNNU.SI,41.071429,67.857143,42.080357,0.280000,0.395000,0.294000,0.308000,0.322000,0.336000,0.350000
2020-04-30,CDL HTrust,J85.SI,42.335766,83.941606,93.678832,0.685000,0.975000,0.719250,0.753500,0.787750,0.822000,0.856250
